In [1]:
import pandas as pd

# Baca file CSV
df = pd.read_csv("sampled_segmentation.csv")

# Ambil kolom 'Game Name' dan hapus duplikat serta nilai kosong
game_list = df['Game Name'].dropna().unique()

# Buat DataFrame baru dari list game
games_df = pd.DataFrame(game_list, columns=["Game Name"])

# Simpan ke file CSV baru
games_df.to_csv("game_list.csv", index=False)

print("Daftar game berhasil disimpan ke 'game_list.csv'")


Daftar game berhasil disimpan ke 'game_list.csv'


In [4]:
import pandas as pd
import glob

# 1. Baca daftar game
game_list = pd.read_csv("game_list.csv")  # asumsi kolom: 'Game Name', 'Genres'

# 2. Gabungkan semua file review
review_files = glob.glob("reviews_1/reviews_processed_data_*.csv") + \
               glob.glob("reviews_2/reviews_processed_data_*.csv")
all_reviews = pd.concat((pd.read_csv(file) for file in review_files), ignore_index=True)

# 3. Normalisasi nama game agar cocok
all_reviews['Game'] = all_reviews['Game'].str.strip()
game_list['Game Name'] = game_list['Game Name'].str.strip()

# 4. Filter review hanya untuk game dalam daftar
filtered_reviews = all_reviews[all_reviews['Game'].isin(game_list['Game Name'])]

# 5. Gabungkan genre dari game_list ke filtered_reviews
merged = filtered_reviews.merge(game_list, how='left', left_on='Game', right_on='Game Name')

# 6. Ambil maksimal 10 review per game
sampled = (
    merged.groupby('Game', sort=False)
    .head(10)
    .reset_index(drop=True)
)

# 7. Pilih kolom akhir dan ubah nama
final_df = sampled[['Game', 'Genre', 'Review']].rename(columns={
    'Game': 'game',
    'Genre': 'genre',
    'Review': 'review'
})

# 8. Jaga urutan game sesuai dengan game_list.csv
final_df['game'] = pd.Categorical(final_df['game'], categories=game_list['Game Name'], ordered=True)
final_df = final_df.sort_values('game').reset_index(drop=True)

# 9. Simpan ke file CSV
final_df.to_csv("sampled_segmentation_reviews.csv", index=False)

print("Review berhasil disimpan ke 'sampled_segmentation_reviews.csv'")


Review berhasil disimpan ke 'sampled_segmentation_reviews.csv'
